In [2]:
import numpy as np
import pandas as pd

In [3]:
def mwu(data):
    #data = np.array(data)
    #data is a T by N matrix with each row being the returns of each expert i at time t
    T = len(data)
    N = len(data.columns)
    epsilon = np.sqrt(np.log(N)/T) # is this correct
    #find bounds for loss
    M = 1
    w = np.ones(N)
    #iterations
    for t in range(T):
        #losses
        loss = -1*data.iloc[t]/M #positive return is negative loss to increase weight
        #update
        for i in range(N):
            w[i] = w[i]*(1-epsilon*loss[i])
        
    p = w/sum(w)
    return p

In [4]:
data = pd.read_excel('trading_data.xlsx')
mwu(data)

array([0.02913591, 0.02708444, 0.02888925, 0.02931947, 0.02715708,
       0.02862985, 0.02952168, 0.02857607, 0.02837603, 0.02877581,
       0.02776614, 0.02958612, 0.027136  , 0.03016261, 0.02860727,
       0.02843283, 0.02834504, 0.0280177 , 0.02908842, 0.02790144,
       0.02816426, 0.02885205, 0.02879602, 0.02877013, 0.02936607,
       0.02770713, 0.02853586, 0.02915484, 0.02756943, 0.02905889,
       0.02832539, 0.02960555, 0.02927283, 0.02854184, 0.02777056])

997

35